### TODO LIST
3. select positive true_cm_g1 and negative true_cm_g2 objects and calculate g_i by formula
4. fit linearly 
5. compare real value
6. upper limit of snr
7. hump
8. select objects with input shape and repeat
9. read covariance, point-correlation function

# Header

In [1]:
import matplotlib
# matplotlib.use("agg")
import matplotlib.pyplot as plt
import pylab
import numpy as np
import h5py
from astropy.io import fits
from tqdm import tqdm


In [2]:
detect = fits.open('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits')[-1]
phot   = fits.open('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits')
id_balrog = fits.open('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits')

#all objects riz
mcal   = h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-v1.2.h5','r')['catalog'] #with neighbor
mcal1  = h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-v1.2.h5','r')['catalog'] #no neighbor
#only balrog objects riz
mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog'] #with neighbor
mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog'] #no neighbor

#only balrog griz
mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog'] #with neighbor
mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog'] #no neighbor

In [3]:
#setting cut threshold 
SNR_CUT = 10
SIZE_RATIO_CUT = 0.5
T_CUT = 1000

In [4]:
# noinspection JupyterKernel,JupyterKernel
log_keys = ['snr','size_ratio','T','T_err','flux_i','flux_r','flux_z','covmat_1_1','covmat_1_2','covmat_2_1','covmat_2_2']
excluded_keys = ['coadd_object_id','flags','bal_id','flags']
excluded_balrog = excluded_keys + []

# Metadata

In [5]:
# FITS format code         Description                     8-bit bytes

# L                        logical (Boolean)               1
# X                        bit                             *
# B                        Unsigned byte                   1
# I                        16-bit integer                  2
# J                        32-bit integer                  4
# K                        64-bit integer                  4
# A                        character                       1
# E                        single precision floating point 4
# D                        double precision floating point 8
# C                        single precision complex        8
# M                        double precision complex        16
# P                        array descriptor                8
# Q                        array descriptor                16

In [6]:
def get_np_unsheared_array(catalog_array):
    catalog_array_unsheared=[]
    for catalog in catalog_array:
        data_dict={}
        for key in list(catalog['unsheared'].keys()):
            data_dict[key]=catalog['unsheared'][key].value
        catalog_array_unsheared.append(data_dict)
    return (catalog_array_unsheared)

In [7]:
catalog_array_unsheared=get_np_unsheared_array(catalog_array)

NameError: name 'catalog_array' is not defined

In [ ]:
name_array = ["mcal","mcal1","mcal2","mcal3","mcal4","mcal5"]
catalog_array = [mcal,mcal1,mcal2,mcal3,mcal4,mcal5]
keys_array = [list(mcal.keys()) for mcal in catalog_array_unsheared]

## Masking function

In [ ]:
def masking(catalog,SNR_CUT,SIZE_RATIO_CUT,T_CUT):
    snr_mask = np.ma.masked_greater(catalog['snr'][:],SNR_CUT)
    size_ratio_mask = np.ma.masked_greater(catalog['size_ratio'][:],SIZE_RATIO_CUT)
    T_mask = np.ma.masked_less(catalog['T'],T_CUT)
    mask_temp = np.logical_and(snr_mask.mask, size_ratio_mask.mask)
    mask = np.logical_and(mask_temp, T_mask.mask)
    return ([snr_mask,size_ratio_mask,T_mask,mask]) #index 3 is the final mask

In [ ]:
def get_mask(catalog_array_unsheared):
    mask_array = []
    for catalog in catalog_array_unsheared:
        mask_array.append(masking(catalog,SNR_CUT,SIZE_RATIO_CUT,T_CUT))
    return (mask_array)

In [ ]:
def print_mask():
    for i,catalog in enumerate(catalog_array_unsheared):
        print ("For {:5}, snr > {:3} mask cuts {:8} elements, size_ratio > {:4} mask cuts {:8} elements, T > {:5} mask cuts {:5} elements".format
          (name_array[i],SNR_CUT,np.sum(~mask_array[i][0].mask),SIZE_RATIO_CUT,np.sum(~mask_array[i][1].mask),T_CUT,np.sum(~mask_array[i][2].mask)))

In [ ]:
mask_array = get_mask(catalog_array_unsheared)

In [ ]:
print_mask()

In [8]:
mask_array = [mask[3] for mask in mask_array]

NameError: name 'mask_array' is not defined

# Std and mean

In [9]:
def mask_catalog_arrays(catalog_array,mask_array):
    masked_catalog = []
    for i,catalog in enumerate(catalog_array):
        data_dict = {}
        for key in list(catalog.keys()):
            data_dict[key] = catalog[key][mask_array[i]]
        masked_catalog.append(data_dict)
    return (masked_catalog)

In [10]:
masked_catalog = mask_catalog_arrays(catalog_array_unsheared,mask_array)

NameError: name 'catalog_array_unsheared' is not defined

In [11]:
for catalog in masked_catalog:
    print(len(list(catalog.keys())))

NameError: name 'masked_catalog' is not defined

In [ ]:
def get_mean_std(masked_catalog):
    std_array=[]; mean_array=[]
    mean_dict={}
    std_dict={}
    for i, catalog in enumerate(masked_catalog):
        std_dict = {}
        mean_dict = {}
        for key in catalog:
            std_dict[key] = np.std(catalog[key])
            mean_dict[key] = np.mean(catalog[key])
        std_array.append(std_dict)
        mean_array.append(mean_dict)
    return(mean_array,std_array)

In [12]:
mean_array,std_array = get_mean_std(masked_catalog)

NameError: name 'get_mean_std' is not defined

In [13]:
f = open('./hist/compare/val_stat.txt','w+')

sepbar = "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n"
f.write ( "std in {:15} is:   {:10} {:10} {:10} {:10} {:10} {:10}| mean is: {:10} {:10} {:10} {:10} {:10} {:10} \n".format("column",*(name_array),*(name_array)))
f.write (sepbar)
for key in list(keys_array[4]):
    if key not in excluded_keys:
        if key in keys_array[0]:
            key_std = lambda key:[std_array[i][key] for i in range(len(std_array))]
            key_mean = lambda key:[mean_array[i][key] for i in range(len(mean_array))]
            f.write ("std in {:15} is: {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e}| mean is: {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e} \n".format(key,*key_std(key),*key_mean(key)))
        else:
            key_std=[None]*len(std_array)
            key_mean=[None]*len(std_array)
            for i in range(len(std_array)):
                if key in std_array[i]:
                    key_std[i] = std_array[i][key]
                    key_mean[i]=mean_array[i][key]
                else:
                    key_std[i]=float("NaN")
                    key_mean[i] = float("NaN")
            f.write ("std in {:15} is: {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e}| mean is: {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e} {:10.3e} \n".format(key,*key_std,*key_mean))
        f.write (sepbar)
f.close()

FileNotFoundError: [Errno 2] No such file or directory: './hist/compare/val_stat.txt'

# Plotting

In [ ]:
import os

def mkdir_p(mypath):    
    try:  
        os.mkdir(mypath)
    except OSError as e:  
        if e.errno != os.errno.EEXIST:
            raise
        pass

In [14]:
def hist_compare(catalog1,catalog2):

    index1 = catalog_array.index(catalog1)
    index2 = catalog_array.index(catalog2)
    
    name1 = name_array[index1]
    name2 = name_array[index2]
    
    for i,key in enumerate(list(catalog1['unsheared'].keys())):
        data1 = catalog1['unsheared'][key][:][mask_array[index1][3]]
        data2= catalog2['unsheared'][key][:][mask_array[index2][3]]

        lower1 = mean_array[index1][key] - 2*std_array[index1][key]
        upper1 = mean_array[index1][key] + 2*std_array[index1][key]
        lower2 = mean_array[index2][key] - 2*std_array[index2][key]
        upper2 = mean_array[index2][key] + 2*std_array[index2][key]
        lower = lower1 if lower1 <= lower2 else lower2;
        upper = upper1 if upper1 >= upper2 else upper2;


        fig = plt.figure();
        ax = fig.add_subplot(111)
        
        if key in excluded_keys:
            continue
        elif key in log_keys:
            lower = 10**(-2) if lower <= 0 else lower;
            BINS = 10 ** np.linspace(np.log10(lower),np.log10(upper))
            H1 = plt.hist(data1,label ='{}'.format(name_array[index1]),bins=BINS,log=True,histtype = 'step');
            H2 = plt.hist(data2,label ='{}'.format(name_array[index2]),bins=BINS,log=True,histtype = 'step');
            plt.xscale('log')
    #     elif key in flux_keys:
    #         data = 30 - 2.5 * np.log10(data)
    #         H = plt.hist(data,label=labels,bins=100)
        else:
            H1 = plt.hist(data1,label ='{}'.format(name1),log=True, bins = 100,histtype = 'step');
            H2 = plt.hist(data2,label ='{}'.format(name2),log=True, bins = 100,histtype = 'step');
        ax.text(1,1,'{:5}:std={:05.3e} mean={:05.3e} \n {:5}:std={:05.3e} mean={:05.3e}'
                .format(name1,std_array[index1][key],mean_array[index1][key],name2,std_array[index2][key],mean_array[index2][key])
                ,verticalalignment = 'top',horizontalalignment = 'right',
        transform=ax.transAxes,color='black',fontsize=7)
        
        plt.xlabel(key)
        plt.ylabel("frequency")
    #     plt.legend(loc=0)
    
        plt.title("Histogram of {}".format(key));
        output_dir = "./hist/compare/{}_{}".format(name1,name2)
        mkdir_p("./hist/compare")
        mkdir_p(output_dir)
        plt.savefig("{}/{}".format(output_dir,key),dpi=300);
        plt.close()

# Calculate g_i

In [15]:
def g(e, R):
    return (e/R)

In [16]:
def get_g_array(mean_array):
    g_array = []
    for i in range(len(mean_array)):
        g_temp_array = []
        for j in range(1,3):
            g_temp_array.append(g(mean_array[i]['e_'+str(j)], mean_array[i]['R'+str(j)+str(j)]))
        g_array.append(g_temp_array)
    return(g_array)

In [17]:
g_array=get_g_array(mean_array)

NameError: name 'mean_array' is not defined

In [18]:
for array in g_array:
    print ("g1: {:10.3e} g2:{:10.3e}".format(*array))

NameError: name 'g_array' is not defined

# Difference comparing

### Testing whether one set is the subset of the other

In [19]:
index_array = []

def get_intersected_mask(catalog1,catalog2):
    
    id_array_1 = catalog1['bal_id'][:]
    id_array_2 = catalog2['bal_id'][:]

    mask1 = np.in1d(id_array_1,id_array_2)
    mask2 = np.in1d(id_array_2,id_array_1)

    if np.sum(mask1)==np.sum(mask2):
        print("The number of elements after taking the intersection of two sets are {}".format(np.sum(mask1)))
        print("The intersection operation takes {} elements from array1".format(len(id_array_1)-np.sum(mask1)))
        print("The intersection operation takes {} elements from array2".format(len(id_array_2)-np.sum(mask2)))

    if np.all((id_array_1[mask1] == id_array_2[mask2])):
        print("The balrog ids are two identical arrays after masking.")
        
    return(mask1,mask2)

In [20]:
mask2,mask3=get_intersected_mask(catalog_array_unsheared[2],catalog_array_unsheared[3])
mask4,mask5=get_intersected_mask(catalog_array_unsheared[4],catalog_array_unsheared[5])

NameError: name 'catalog_array_unsheared' is not defined

In [21]:
bal_mask_array=[mask2,mask3,mask4,mask5]
bal_catalog_array=catalog_array_unsheared[2:6]

NameError: name 'mask2' is not defined

In [22]:
def get_bal_masked_dict(old_dict,mask):
    masked_dict={}
    for key in tqdm(old_dict.keys()):
        masked_dict[key]=old_dict[key][mask]
    return (masked_dict)

In [23]:
catalog_bal_masked=[]
for i in range(len(bal_mask_array)):
    catalog_bal_masked.append(get_bal_masked_dict(bal_catalog_array[i],bal_mask_array[i]))

NameError: name 'bal_mask_array' is not defined

In [24]:
print (len(catalog_bal_masked[0]['bal_id']))
print (len(catalog_bal_masked[1]['bal_id']))
print (len(catalog_bal_masked[2]['bal_id']))
print (len(catalog_bal_masked[3]['bal_id']))

IndexError: list index out of range

### clipping the arrays with logical and

In [25]:
SNR_mask_array_for_bal_masked = [masking(catalog,SNR_CUT,SIZE_RATIO_CUT,T_CUT) for catalog in catalog_bal_masked]

In [26]:
print (len(SNR_mask_array_for_bal_masked[0][3]))
print (len(SNR_mask_array_for_bal_masked[1][3]))
print (len(SNR_mask_array_for_bal_masked[2][3]))
print (len(SNR_mask_array_for_bal_masked[3][3]))

IndexError: list index out of range

In [27]:
def get_and_mask(catalog_array,mask_array):
    and_masked_array=[]
    mask_2_3 = np.logical_and(mask_array[0][3],mask_array[1][3])
    mask_4_5=np.logical_and(mask_array[2][3],mask_array[3][3])
    for i in range(2):
        and_masked_array.append(mask_2_3)
    for i in range(2):
        and_masked_array.append(mask_4_5)
    return (and_masked_array)
        

In [28]:
print (len(and_mask_array))

NameError: name 'and_mask_array' is not defined

In [29]:
and_mask_array=get_and_mask(catalog_bal_masked,SNR_mask_array_for_bal_masked)

IndexError: list index out of range

In [30]:
final_catalog_array=mask_catalog_arrays(catalog_bal_masked,and_mask_array)

NameError: name 'and_mask_array' is not defined

In [31]:
def get_difference(catalog_w,catalog_wo,keys):
    diff_dict = {}
    frac_diff = {}

    for key in list(keys):
        diff_dict[key] = catalog_wo[key]-catalog_w[key]

    for key in list(keys):
        frac_diff[key] = np.array(diff_dict[key])/((np.array(catalog_wo[key])+np.array(catalog_w[key])/2))

    mean_diff = {}
    std_diff = {}
    
    for key in list(keys):
        std_diff[key] = np.std(diff_dict[key])
        mean_diff[key]= np.mean(diff_dict[key])

    std_frac = {}
    mean_frac = {}
    
    for key in list(keys):
        std_frac[key] = np.std(frac_diff[key])
        mean_frac[key]= np.mean(frac_diff[key])
        
    return(diff_dict,frac_diff,mean_diff,std_diff,std_frac,mean_frac)

### Plotting the absolute difference histogram

In [32]:
def diff_plot (name, diff_dict,mean_diff,std_diff,keys):
    abs_name = name
    for i,key in enumerate(keys):

    #     print (" {:05.2f}%".format(i*100/len(mcal_keys)),sep=' ', end='',flush=True)
        data = diff_dict[key][:]

        lower = mean_diff[key] - 2*std_diff[key]
        upper = mean_diff[key] + 2*std_diff[key]

        fig = plt.figure();
        ax = fig.add_subplot(111);

        if key in excluded_balrog:
            continue
        elif key in log_keys:
            lower = 10**(-2) if lower <= 0 else lower;
            BINS = 10 ** np.linspace(np.log10(lower),np.log10(upper))
            H = plt.hist(data,label ='{}'.format(abs_name),bins=BINS,log=True,histtype = 'step');
            plt.xscale('log');
            plt.xlabel(key+' in log scale');
        else:
            H = plt.hist(data,label ='{}'.format(abs_name),log=True, bins = 50,histtype = 'step');
            plt.xlabel(key + " in linear scale");
        ax.text(1,1,'std={:05.3e} mean={:05.3e}'
                .format(std_diff[key],mean_diff[key]),verticalalignment = 'top',horizontalalignment = 'right',transform=ax.transAxes,color='black',fontsize=7);
        plt.ylabel("frequency");
        plt.title("Histogram of difference in {}".format(key));
        mkdir_p("./hist/diff")
        output_dir = "./hist/diff/{}".format(abs_name);
        mkdir_p(output_dir);
        plt.savefig("{}/{}".format(output_dir,key),dpi=300);
        plt.close()

In [33]:
diff_plot("riz_abs_diff",riz_diff,riz_mean_diff,riz_std_diff,mcal1_keys)
diff_plot("griz_abs_diff",griz_diff,griz_mean_diff,griz_std_diff,mcal_keys)

NameError: name 'riz_diff' is not defined

In [34]:
diff_plot("riz_frac_diff",riz_frac,riz_mean_frac,riz_std_frac,mcal1_keys)
diff_plot("griz_frac_diff",griz_frac,griz_mean_frac,griz_std_frac,mcal_keys)

NameError: name 'riz_frac' is not defined

In [35]:
f = open('./hist/diff/diff_stat.txt','w+')

f.write ("{:15}      {:>15} {:>15} {:>15} {:>15} | {:>15} {:>15} {:>15} {:>15} \n".format("NAME", "abs_diff_riz", "frac_diff_riz", "abs_diff_griz","frac_diff_griz","abs_diff_riz", "frac_diff_riz", "abs_diff_griz","frac_diff_griz"  ))
f.write (sepbar)
for key in list(mcal_keys):
    if key not in excluded_keys:
        if key in mcal1_keys:
            f.write ("{:15}      {:15.3e} {:15.3e} {:15.3e} {:15.3e} | {:15.3e} {:15.3e} {:15.3e} {:15.3e} \n".format(key,riz_std_diff[key], riz_std_frac[key], griz_std_diff[key],griz_std_frac[key],riz_mean_diff[key], riz_mean_frac[key], griz_mean_diff[key],griz_mean_frac[key] ))
            f.write(sepbar)
        else:
#             key_std=[None]*len(std_array)
#             key_mean=[None]*len(std_array)
#             for i in range(len(std_array)):
#                 if key in std_array[i]:
#                     key_std[i] = std_array[i][key]
#                     key_mean[i]=mean_array[i][key]
#                 else:
#                     key_std[i]=float("NaN")
#                     key_mean[i] = float("NaN")
            f.write ("{:15}      {:15.3e} {:15.3e} {:15.3e} {:15.3e} | {:15.3e} {:15.3e} {:15.3e} {:15.3e} \n".format(key,float("NaN"),float("NaN") , griz_std_diff[key],griz_std_frac[key],float("NaN"), float("NaN"), griz_mean_diff[key],griz_mean_frac[key] ))
            f.write (sepbar)
f.close()




FileNotFoundError: [Errno 2] No such file or directory: './hist/diff/diff_stat.txt'